In [97]:
import pandas as pd
from sklearn import model_selection
import numpy as np
import sklearn.metrics
from sklearn.model_selection import train_test_split  
import lightgbm as lgb
from sklearn import preprocessing

In [114]:
print('开始处理特征...')
train_path = 'data/train_and_test/train.csv'
test_path = 'data/train_and_test/test.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
used_feature = ['create_count', 'create_day_mean', 'create_day_max',
       'create_day_min', 'create_day_std', 'create_day_var',
       'last_day_cut_max_day', 'create_sub_register_x',
       'max_create_times_per_day', 'create_day_diff_mean',
       'create_day_diff_max', 'create_day_diff_min', 'create_day_diff_std',
       'create_day_diff_var', 'device_type', 'register_type',
       'register_day_cut_max_day', 'launch_count', 'launch_day_mean',
       'launch_day_max', 'launch_day_min', 'launch_day_std', 'launch_day_var',
       'launch_day_cut_max_day', 'create_sub_register_y',
       'max_launch_times_per_day', 'launch_day_diff_mean',
       'launch_day_diff_max', 'launch_day_diff_min', 'launch_day_diff_std',
       'launch_day_diff_var', 'activity_count', 'activity_day_mean',
       'activity_day_max', 'activity_day_min', 'activity_day_std',
       'activity_day_var', 'activity_day_ske', 'activity_day_kur',
       'activity_day_cut_max_day', 'create_sub_register', '0_page_count',
       '1_page_count', '2_page_count', '3_page_count', '4_page_count',
       '0_page_count_div_sum', '1_page_count_div_sum', '2_page_count_div_sum',
       '3_page_count_div_sum', '4_page_count_div_sum', '0_action_count',
       '1_action_count', '2_action_count', '3_action_count', '4_action_count',
       '5_action_count', '0_action_count_div_sum', '1_action_count_div_sum',
       '2_action_count_div_sum', '3_action_count_div_sum',
       '4_action_count_div_sum', '5_action_count_div_sum',
       'max_act_times_per_day', 
#                 'max_hot_vid', 
#                 'max_hot_auth',
                'mean_hot_vid',
       'mean_hot_auth',
#                 'min_hot_vid',
#                 'min_hot_auth',
#                 'sum_hot_vid',
#        'sum_hot_auth', 
                'activity_day_diff_mean', 'activity_day_diff_max',
       'activity_day_diff_min', 'activity_day_diff_std',
       'activity_day_diff_var', 'activity_day_diff_ske',
       'activity_day_diff_kur', 'watch_self']
used_feature = np.array(used_feature)
importance_feature = [6, 17, 18, 19, 21, 23, 26, 28, 29, 31, 33, 37, 39, 42, 44, 46, 47, 49, 51, 57, 58, 63, 64, 65, 69, 71, 72]
used_feature = used_feature[np.array(importance_feature)]
print(used_feature)
train_feature = train[used_feature]
test_feature = test[used_feature]
label = train['label']
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(train_feature, label, test_size=0.3,random_state=1024)


开始处理特征...
['last_day_cut_max_day' 'launch_count' 'launch_day_mean' 'launch_day_max'
 'launch_day_std' 'launch_day_cut_max_day' 'launch_day_diff_mean'
 'launch_day_diff_min' 'launch_day_diff_std' 'activity_count'
 'activity_day_max' 'activity_day_ske' 'activity_day_cut_max_day'
 '1_page_count' '3_page_count' '0_page_count_div_sum'
 '1_page_count_div_sum' '3_page_count_div_sum' '0_action_count'
 '0_action_count_div_sum' '1_action_count_div_sum' 'max_act_times_per_day'
 'mean_hot_vid' 'mean_hot_auth' 'activity_day_diff_std'
 'activity_day_diff_ske' 'activity_day_diff_kur']


In [119]:
print('载入数据......')
lgb_train = lgb.Dataset(train_feature, label)
lgb_eval = lgb.Dataset(X_test, Y_test, reference=lgb_train)


print('开始训练......')
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc', 'binary_logloss'},
}
# params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'regression',
#     'metric': {'l2'}
# }

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=60,
                valid_sets=lgb_eval
                )
gbm.save_model('model/lgb_model.txt')

temp = gbm.predict(X_test)
temp[temp>=0.42]=1
temp[temp<0.42]= 0
print('结果：' + str(sklearn.metrics.f1_score(Y_test, temp)))
print('特征重要性：'+ str(list(gbm.feature_importance())))

载入数据......
开始训练......
[1]	valid_0's binary_logloss: 0.652302	valid_0's auc: 0.870989
[2]	valid_0's binary_logloss: 0.619595	valid_0's auc: 0.873164
[3]	valid_0's binary_logloss: 0.592538	valid_0's auc: 0.873422
[4]	valid_0's binary_logloss: 0.569935	valid_0's auc: 0.873754
[5]	valid_0's binary_logloss: 0.550973	valid_0's auc: 0.874023
[6]	valid_0's binary_logloss: 0.534843	valid_0's auc: 0.874345
[7]	valid_0's binary_logloss: 0.52118	valid_0's auc: 0.874555
[8]	valid_0's binary_logloss: 0.50959	valid_0's auc: 0.874609
[9]	valid_0's binary_logloss: 0.499457	valid_0's auc: 0.875264
[10]	valid_0's binary_logloss: 0.490861	valid_0's auc: 0.875474
[11]	valid_0's binary_logloss: 0.483439	valid_0's auc: 0.875724
[12]	valid_0's binary_logloss: 0.47708	valid_0's auc: 0.875843
[13]	valid_0's binary_logloss: 0.471601	valid_0's auc: 0.876053
[14]	valid_0's binary_logloss: 0.466737	valid_0's auc: 0.876577
[15]	valid_0's binary_logloss: 0.462623	valid_0's auc: 0.876754
[16]	valid_0's binary_logloss:

In [81]:
pre = gbm.predict(test_feature)
df_result = pd.DataFrame()
df_result['user_id'] = test['user_id']
df_result['result'] = pre
df_result.to_csv('result/lgb_result.csv', index=False)
pre[pre >= 0.44]=1
pre[pre < 0.44]=0
pre = map(int,pre)
print('为1的个数：' + str(len(np.where(np.array(pre)==1)[0])))
print('为0的个数：' + str(len(np.where(np.array(pre)==0)[0])))

为1的个数：0
为0的个数：0


In [91]:
lgb = pd.read_csv('result/lgb_result.csv')
res = lgb[lgb['result'] >= 0.44]
print(len(res))
del res['result']
res.to_csv('result2/dealed_result06128164.csv', index=False, header=False)

23569


In [71]:
a = np.array([12, 21, 13, 10, 10, 0, 26, 14, 1, 10, 9, 6, 6, 0, 0, 0, 0, 125, 29, 43, 14, 88, 3, 183, 18, 0, 103, 16, 30, 90, 7, 27, 18, 45, 10, 19, 2, 45, 0, 42, 0, 21, 30, 19, 45, 6, 27, 41, 16, 35, 17, 32, 22, 6, 3, 0, 0, 37, 43, 25, 14, 0, 0, 41, 56, 73, 23, 5, 14, 36, 4, 45, 59, 0])
a.size

74

In [113]:
b = []
for i in range(0, a.size-1):
    if a[i] > 25:
        b.append(i)
print(b)

[6, 17, 18, 19, 21, 23, 26, 28, 29, 31, 33, 37, 39, 42, 44, 46, 47, 49, 51, 57, 58, 63, 64, 65, 69, 71, 72]


In [ ]:
from sklearn.linear_model import LogisticRegression   
  
classifier = LogisticRegression()  # 使用类，参数全是默认的  
classifier.fit(X_train,X_test)  # 训练数据来学习，不需要返回值  
classifier.predict(test_feature)


In [132]:
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans


In [138]:
X_train, X_test, y_train, y_test = train_test_split(train_feature, label, test_size=0.2, random_state=0)
clf = KMeans(n_clusters=2,max_iter=300,n_init=10)
clf.fit(X_train)
ypred = clf.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
rfc = RandomForestClassifier()

In [61]:
len(b)

58